# Variables de entorno

In [1]:
# global scale, nconf, nchip, max_iter, n_pos, t_ext, tmax_chip, t_delta, tam
# global chip_info
# global input_filepath
global objective_function_HEAT

# Leer problema
Definir las funciones para leer la instancia del problema, y para escribir el fichero de input para el código C

In [2]:
import numpy as np

# Lee los parámetros desde fichero
# Devuelve (instance,initial_solution) donde
## instance = scale,nconf,nchip,max_iter,n_pos,t_ext,tmax_chip,t_delta,tam,chip_info
## initial_solution puede ser vacío
def read_params(filepath):    
    # Lee los parámetros de la cabecera
    file = open(filepath,'r')
    params = file.readline().split()
    scale    = int(params[0])
    nchip    = int(params[1])
    max_iter = int(params[5])
    n_pos    = int(params[6])
    nconf    = 1
    t_ext     = float(params[2])
    tmax_chip = float(params[3])
    t_delta   = float(params[4])
    tam = (100*scale)//n_pos
    file.readline()
    
    initial_solution = np.zeros(nchip*2,dtype=np.int32)
    chip_info = np.zeros(nchip*3,dtype=np.int32)
    
    # Lee la información de los chips
    for i in range(nchip):
        chip_info_line = file.readline().split()
        h = int(chip_info_line[0])
        w = int(chip_info_line[1])
        tchip = float(chip_info_line[2])
        chip_info[i*3]   = h
        chip_info[i*3+1] = w
        chip_info[i*3+2] = tchip
        
    # print(chip_info)
    line = file.readline()
    
    # Si hay una solución inicial, la lee
    if not line:
        print("Sin solución inicial")
    else:
        for i in range(nchip):
            chip_position = file.readline().split()
            x = int(chip_position[0])
            y = int(chip_position[1])
            initial_solution[i*2]   = x
            initial_solution[i*2+1] = y
    
    # print(initial_solution)
    instance = scale,nconf,nchip,max_iter,n_pos,t_ext,tmax_chip,t_delta,tam,chip_info
    return instance,initial_solution

# Escribe el fichero de input para el código C 
# (No se debería usar esta función)
def write_card_file(filepath, instance, solution):
    scale,nconf,nchip,max_iter,n_pos,t_ext,tmax_chip,t_delta,tam, chip_info = instance
    
    file = open(filepath,'w')
    
    # Escribir los ficheros cabecera
    file.write("{} {} {} {} {} {} {}\n\n".format(scale,nconf,nchip,t_ext,tmax_chip,t_delta,max_iter))
    
    # Escribe la información de los chips
    for (h,w,tchip) in chip_info:
        file.write("{} {} {}\n".format(h*tam,w*tam,tchip))
    
    file.write("\n")
    
    # Escribe la configuración o solución dada
    for (x,y) in solution:
        file.write("{} {}\n".format(x*tam,y*tam))
    file.write("\n")

# Función objetivo
Preparar la librería para hacer la llamada a la función objetivo  
Crear función de constraint  
Ejecutar un ejemplo  
  

In [3]:
import ctypes

def initialize_fitness():
    # Cargar la librería 
    objective_function_HEAT = ctypes.CDLL("lib/fitness.so")

    # Definir el tipo de los argumentos que se le pasan (relevante) y el del resultado (irrelevante)
    #objective_function_HEAT.fitness.argtypes=  save_results   card_file,      scale         nchip         t_ext          tmax_chip       t_delta          max_iter
    #                                           chip_info                     chip_positions                Tmean_out                       Tej_out)
    objective_function_HEAT.fitness.argtypes = [ctypes.c_int, ctypes.c_char_p, ctypes.c_int, ctypes.c_int, ctypes.c_float, ctypes.c_float, ctypes.c_float, ctypes.c_int,
                                                ctypes.POINTER(ctypes.c_int), ctypes.POINTER(ctypes.c_int), ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_double)]
    
    objective_function_HEAT.fitness.restype = ctypes.c_int
    return objective_function_HEAT
    
def fitness_heat(objective_function_HEAT, filepath, instance, solucion, salida = False):    
    # Preparar las variables de argumento
    if not salida:
        input_int = 0
    else:
        input_int = 1
        
    input_string = filepath.encode('utf-8')  # Necesario convertirlo a char* para C
    
    scale,nconf,nchip,max_iter,n_pos,t_ext,tmax_chip,t_delta,tam,chip_info = instance
    
    guardar_result = ctypes.c_int(input_int)
    fichero_result = ctypes.c_char_p(input_string)
    scale = ctypes.c_int(scale)
    nchip = ctypes.c_int(nchip)
    t_ext = ctypes.c_float(t_ext)
    tmax_chip = ctypes.c_float(tmax_chip)
    t_delta = ctypes.c_float(t_delta)
    max_iter = ctypes.c_int(max_iter)
    
    # Crear arrays de ctypes para chip_info y solucion
    chip_info_array = (ctypes.c_int * len(chip_info))(*chip_info)
    solucion_array = (ctypes.c_int * len(solucion))(*solucion)
    
    # Crear variables ctypes para Tmean y Tej
    Tmean = ctypes.c_double()
    Tej = ctypes.c_double()

    # Ejecución
    resultado = objective_function_HEAT.fitness(
        guardar_result, fichero_result, scale, nchip, t_ext, tmax_chip, t_delta, max_iter,
        chip_info_array, solucion_array, ctypes.byref(Tmean), ctypes.byref(Tej)
    )
    
    return resultado,Tmean.value,Tej.value

def valid_solution(instance, solution):
    scale,nconf,nchip,max_iter,n_pos,t_ext,tmax_chip,t_delta,tam,chip_info = instance
    
    if np.all(solution == 0):
        return False
    
    n_filas = n_pos*2
    n_columnas = n_pos

    # Verificar que ningún chip se salga de la placa
    for i in range(nchip):
        x = solution[i*2]
        y = solution[i*2+1]
        ancho = chip_info[i*3]
        alto  = chip_info[i*3+1]

        if x < 0 or x + ancho > n_filas or y < 0 or y + alto > n_columnas:
            return False

    # Verificar que ningún chip solape con otro chip
    for i in range(nchip - 1):
        x1 = solution[i*2]
        y1 = solution[i*2+1]
        ancho1 = chip_info[i*3]
        alto1  = chip_info[i*3+1]

        for j in range(i + 1, nchip):
            x2 = solution[j*2]
            y2 = solution[j*2+1]
            ancho2 = chip_info[j*3]
            alto2  = chip_info[j*3+1]

            if (x1 < x2 + ancho2 and x1 + ancho1 > x2 and
                y1 < y2 + alto2 and y1 + alto1 > y2):
                return False 

    return True  

input_filepath = "data/input1"
instance, initial_solution = read_params(input_filepath)

if valid_solution(instance,initial_solution):
    output_filepath = "generated_files/card"
    initialize_fitness()
    result_valid, fitness_value, fitness_time = fitness_heat(output_filepath,instance,initial_solution)

    if not result_valid:
        print("Valor fitness: ",fitness_value)
        print("Ha tardado {} segundos".format(fitness_time))
    else:
        print("Program ended with error")
else:
    print("Not a valid solution")

Valor fitness:  36.520565032958984
Ha tardado 17.469277696 segundos


# Funciones para calidad de vida

In [4]:
import matplotlib.pyplot as plt

def visualizar_solucion(instance, solution):
    scale,nconf,nchip,max_iter,n_pos,t_ext,tmax_chip,t_delta,tam,chip_info = instance
    
    # Crear la figura y los ejes
    fig, ax = plt.subplots(figsize=(20, 10))  

    # Dibujar la placa
    for i in range(n_pos*2 + 1):
        y = i * tam
        ax.plot([0, scale * 100], [y, y], color='gray', linestyle='--', linewidth=0.5)

    for i in range(n_pos + 1):
        x = i * tam
        ax.plot([x, x], [0, scale * 200], color='gray', linestyle='--', linewidth=0.5)

    # Dibujar los chips en la placa
    for i in range(nchip):
        # Invertir las coordenadas
        y = solution[i*2] 
        x = solution[i*2+1]
        x_real = x * tam  # Usar alto_pos para el eje horizontal
        y_real = y * tam  # Usar ancho_pos para el eje vertical
        
        # Invertir las dimensiones
        alto = chip_info[i*3]
        ancho = chip_info[i*3+1]
        temperatura = chip_info[i*3+2]
        alto_real = alto * tam
        ancho_real = ancho * tam

        # Asignar un color diferente a cada chip
        color = plt.colormaps.get_cmap('tab10')(i)

        # Dibujar el rectángulo con el color y la etiqueta
        rect = plt.Rectangle((x_real, y_real), ancho_real, alto_real, linewidth=1, edgecolor='black', facecolor=color)
        ax.add_patch(rect)

        # Mostrar el nombre del chip y su temperatura 
        texto = f'C{i+1} - {temperatura}°C'
        ax.text(x_real + ancho_real / 2, y_real + alto_real / 2, texto, color='black',
                ha='center', va='center', fontsize=8, fontweight='bold')

    # Etiquetas y título
    ax.set_xlabel('Alto de la placa - x')  # Cambiar las etiquetas
    ax.set_ylabel('Ancho de la placa - y')
    ax.set_title("Placa del fichero {}".format(input_filepath))

    # Mostrar la placa
    plt.show()

def print_general_params(instance):
    scale,nconf,nchip,max_iter,n_pos,t_ext,tmax_chip,t_delta,tam,chip_info = instance
    
    print("scale is: {}".format(scale))
    print("nconf is: {}".format(nconf))
    print("nchip is: {}".format(nchip))
    print("t_ext is: {}".format(t_ext))
    print("tmax_chip is: {}".format(tmax_chip))
    print("t_delta is: {}".format(t_delta))
    print("max_iter is: {}".format(max_iter))
    print("n_pos is: {}".format(n_pos))
    print("tam is: {}".format(tam))
    print("chip_info is: ",chip_info)

# Ejecución con solución inicial

In [5]:
input_filepath = "data/input1"
output_filepath = 'generated_files/card1'
instance, initial_solution = read_params(input_filepath)

if valid_solution(instance,initial_solution):
    initialize_fitness()
    result_valid, fitness_value, fitness_time = fitness_heat(output_filepath,instance,initial_solution)

    if not result_valid:
        print("Valor fitness: ",fitness_value)
        print("Ha tardado {} segundos".format(fitness_time))
        visualizar_solucion(instance,initial_solution)
    else:
        print("Program ended with error")
else:
    print("Not a valid solution")

: 

: 

# Solución inicial estocástica y vecindades

In [ ]:
import random as rm

def random_solution(instance):
    scale,nconf,nchip,max_iter,n_pos,t_ext,tmax_chip,t_delta,tam,chip_info = instance
    
    solution = np.empty(nchip*2,dtype=np.int32)
    
    valid = False
    while not valid:
        
        for i in range(nchip):
            x = rm.randint(0,n_pos*2)
            y = rm.randint(0,n_pos)
            
            solution[i*2]   = x
            solution[i*2+1] = y
            
        valid = valid_solution(instance,solution)
            
    return solution

rand_sol = random_solution(instance)
print(rand_sol)
visualizar_solucion(instance,rand_sol)    
        

# Función de vecindad 
Dos funciones de vecindad  
1. Mueve un chip una posición en cualquiera de los 8 sentidos  
2. Swap entre dos chips  

In [ ]:
def move_1 (instance, solucion):
    scale,nconf,nchip,max_iter,n_pos,t_ext,tmax_chip,t_delta,tam,chip_info = instance
    result = np.empty(0)
    
    for i in range(nchip):
        x = solucion[2*i]
        y = solucion[2*i+1]
        
        vecino_0 = solucion.copy()
        vecino_1 = solucion.copy()
        vecino_2 = solucion.copy()
        vecino_3 = solucion.copy()
        vecino_4 = solucion.copy()
        vecino_5 = solucion.copy()
        vecino_6 = solucion.copy()
        vecino_7 = solucion.copy()
        
        # Fila superior
        vecino_0[2*i] = (x-1)%(n_pos*2); vecino_0[2*i+1] = (y-1)%n_pos
        vecino_1[2*i] = x              ; vecino_1[2*i+1] = (y-1)%n_pos
        vecino_2[2*i] = (x+1)%(n_pos*2); vecino_2[2*i+1] = (y-1)%n_pos
        
        # Fila media
        vecino_3[2*i] = (x-1)%(n_pos*2); vecino_3[2*i+1] = y
        vecino_4[2*i] = (x+1)%(n_pos*2); vecino_4[2*i+1] = y
        
        # Fila inferior
        vecino_5[2*i] = (x-1)%(n_pos*2); vecino_5[2*i+1] = (y+1)%n_pos
        vecino_6[2*i] = x              ; vecino_6[2*i+1] = (y+1)%n_pos
        vecino_7[2*i] = (x+1)%(n_pos*2); vecino_7[2*i+1] = (y+1)%n_pos
        
        result = np.append(result,[vecino_0.copy(),vecino_1.copy(),vecino_2.copy(),vecino_3.copy(),vecino_4.copy(),vecino_5.copy(),vecino_6.copy(),vecino_7.copy()])
  
    result = result.reshape(len(result)//(nchip*2),nchip*2)  
    return result

def swap_2(instance, solucion):
    scale,nconf,nchip,max_iter,n_pos,t_ext,tmax_chip,t_delta,tam,chip_info = instance
    result = np.empty(0)

    for i in range(nchip):
        for j in range(i + 1, nchip):
            vecino_swap = solucion.copy()

            # Intercambiar la x entre los chips i y j
            vecino_swap[2*i], vecino_swap[2*j] = vecino_swap[2*j], vecino_swap[2*i]
            
            # Intercambiar la y entre los chips i y j
            vecino_swap[2*i+1], vecino_swap[2*j+1] = vecino_swap[2*j+1], vecino_swap[2*i+1]

            # Verificar si la solución resultante es válida
            if valid_solution(instance,vecino_swap):
                result = np.append(result,vecino_swap.copy())

    result = result.reshape(len(result)//(nchip*2),nchip*2)  
    return result

rand_sol = random_solution(instance)
print("Solución inicial aleatoria:",rand_sol)
# visualizar_solucion(rand_sol[0]) 
   
vecindad_1 = move_1(instance,rand_sol)
# print("Vecindad move_1:",vecindad_1)
print("Tam vecindad move_1:",len(vecindad_1))
# for elem in vecindad_1[0]:
#     visualizar_solucion(elem)  

vecindad_2 = swap_2(instance,rand_sol)
# print("Vecindad swap_2:",vecindad_2)
print("Tam vecindad swap_2:",len(vecindad_2))  
# for elem in vecindad_2[0]:
#     visualizar_solucion(elem)  

# Búsquedas baseline 
Random Search  
Brute force (?)

In [ ]:
import sys

def random_search(instance, num_solutions) -> list:
    scale,nconf,nchip,max_iter,n_pos,t_ext,tmax_chip,t_delta,tam,chip_info = instance
    
    best_sol_codif = random_solution(instance)
    _, best_sol_value, total_time = fitness_heat("",instance,best_sol_codif,salida=False)

    sol_actual_codif = best_sol_codif.copy()
    sol_actual_value = best_sol_value

    for _ in range(num_solutions):
        sol_actual_codif = random_solution(instance)
        _, sol_actual_value, sol_actual_time = fitness_heat("",instance,sol_actual_codif,salida=False)
        total_time += sol_actual_time
        if sol_actual_value < best_sol_value:
            best_sol_value = sol_actual_value
            best_sol_codif = sol_actual_codif

    return best_sol_value,best_sol_codif, total_time

best_sol_value, best_sol_codif, total_time = random_search(instance,1000)
print("Best fitness:",best_sol_value)
print("Best fitness codif:",best_sol_codif)
print("Total time:",total_time)
visualizar_solucion(best_sol_codif)

# Basic Hill Climbing - Búsqueda Local

In [ ]:
def best_first_move(instance: list, candidato: list, fitness_candidato : float, max_eval : int, n_eval : int):
    current_sol = candidato.copy()
    current_fitness = fitness_candidato

    iter_count = n_eval
    mejora = True
    total_time = 0

    # Mientras no sobrepasen las iteraciones o no se encuentre mejora
    while (iter_count < max_eval and mejora):
      # Se obtiene la vecindad
      vecindad = move_1(current_sol)

      # Se recorren los vecinos hasta encontrar uno con mejor solucion
      sol_ind = 0
      encontrado = False

      # Se recorren los vecinos hasta encontrar el primero que mejore la función fitness
      while (sol_ind < len(vecindad[0]) and not encontrado):
        if (iter_count > max_eval):
          break
        # Se calcula el fitness de uno de los vecinos
        write_card_file(filepath="card_tmp",solution=[vecindad[0][sol_ind]])
        new_fitness, new_time = fitness_heat(filepath="card_tmp",salida=False)
        total_time += new_time
        iter_count = iter_count + 1
        # Si mejora el fitness se sigue ese camino
        if (new_fitness < current_fitness):
          print("Fitness mejorado de {} a {}".format(current_fitness,new_fitness))
          current_fitness = new_fitness
          current_sol = vecindad[sol_ind]
          encontrado = True
        sol_ind = sol_ind + 1

      # Si no se ha encontrado un vecino que mejore, no merece seguir buscando
      if not encontrado:
        mejora = False

    best_fitness = current_fitness
    best_solution = current_sol
    return best_fitness,best_solution,iter_count, total_time
  
# Preparar los datos
instance = nchip, n_pos
candidato = random_solution(nchip,n_pos)
write_card_file(filepath="card_tmp",solution=candidato)
fitness_candidato, _ = fitness_heat(filepath="card_tmp",salida=False)
max_eval = 1000
n_eval = 1

# Hacer la búsqueda local
best_sol_value, best_sol_codif, iter_count, total_time = best_first_move(instance,candidato,fitness_candidato,max_eval,n_eval)

# Comprobar los resultados
print("Best fitness:",best_sol_value)
print("Best fitness codif:",best_sol_codif)
print("Iterations used:",iter_count)
print("Total time:",total_time)
visualizar_solucion(best_sol_codif[0])
